# Imports

In [1]:
import numpy as np
from IPython.core.pylabtools import figsize
from matplotlib import pyplot as plt

from src.algorithms import AbstractMetropolisHastings

figsize(20, 3)

# VMC Setup

The goal of this section is to calculate $\int ... dR $

$$
\left \{
\begin{array}{rll}
H &= - \frac{\partial^2}{2 \partial x^2} +V(x) & \text{Hamiltonian}\\
\psi(R) &= \exp({-p_0(x-p_1)^2-p_2(x-p_3)^2}) & \text{Wave function}\\
E_L(R) &= \frac{H\psi(R)}{\psi(R)} = V(x) - \frac{1}{2}\frac{\partial^2}{\partial x^2}\psi(x) & \text{"Local" energy}\\
V(x) &= \frac{a_2}{2}x^2 + a_4x^4 & \text{Potential energy}\\
G(R,R', \beta) &= \braket{R | e^{- \beta H} | R'} & \text{???}\\
\Pi(R) &\propto | \psi(R) |^2 & \text{Sampling probability}\\
\end{array}
\right .
$$

In [2]:
class VMC(AbstractMetropolisHastings[float]):
    a = [1, 0]
    p = [.0002, 0, 0, 0]
    sigma = 5

    def generator_function(self):
        return self.current_configuration + np.random.normal(0, self.sigma)

    def state_likelihood(self, x: float):
        # This is proportional to the state probability
        p = self.p
        aux_exp = lambda mu, sigma: np.prod([-1, mu, x - sigma, x - sigma])
        return np.exp(aux_exp(p[0], p[1]) + aux_exp(p[2], p[3]))


In [3]:
vmc = VMC(initial_configuration=np.random.normal(0, 1))

# Likelihood plot

In [4]:
datapoints = np.linspace(-250, 250, 10000)
plt.plot(datapoints, [vmc.state_likelihood(x) for x in datapoints])

# Trial run

In [5]:
vmc.run_iterations(10 ** 7)

# Plots

In [8]:
plt.plot(vmc.configuration_history)

In [6]:
# 4*10**5 is used to put both plots in the same scale.
# I'd like a neater way of lining the 2 plots
plt.plot(datapoints, [4 * 10 ** 5 * vmc.state_likelihood(x) for x in datapoints])
plt.hist(vmc.configuration_history, bins=100, density=False)